# Getting started with diffpy.mpdf

The diffpy.mpdf pacakge aims to provide a flexible, object-oriented framework for computing the magnetic PDF from magnetic structures and performing refinements against neutron scattering data. This notebook gives an overview of how the mPDF software is organized and provides introductory examples of computing mPDFs. Additional examples are included on the CMI Exchange.

# Contents of this tutorial

- Class structure of the mpdf package
- Simple example: Calculating the mPDF from a spin dimer
- Creating a magnetic structure from a CIF file using diffpy.Structure
- Creating a magnetic structure by defining your own unit cell
- Creating a magnetic structure with multiple species: Simple ferrimagnet
- Creating more complex magnetic structures: One-dimensional spin helix
- Exploring some of the additional parameters in the mPDF
- Simple example of an mPDF fit

# Class structure of the mpdf package

The mpdf package contains several functions and three classes to aid in the mPDF calculations--the MPDFcalculator class, the MagStructure class, and the optional (but often very useful) MagSpecies class.

### MagStructure class

The main job of a MagStructure object is to contain a numpy array of atomic positions (MagStructure.atoms) and a corresponding numpy array of magnetic moment vectors (MagStructure.spins), which are the only two absolutely required ingredients for calculating the mPDF. Additional optional information such as the Lande g-factor or magnetic form factor can be stored in a MagStructure object if it is needed (as it often is for performing mPDF refinements). A diffpy.Structure object corresponding to the atomic structure of the material can also be stored for convenience when generating the atomic positions and spin vectors. Multiple types of magnetic species (e.g. spins on different ionic sites that may have different moment sizes) can be stored inside a single MagStructure object.

### MagSpecies class

The MagSpecies class is intended to provide an easy way to generate the atomic positions and spin vectors of a particular type of magnetic species in a structure. It takes a diffpy.Structure object and packages additional information pertaining to the magnetic structure, such as the magnetic propagation vector(s), magnetic basis vector(s), spin/orbital quantum numbers, and magnetic form factor. Alternatively, the user can define a unit cell populated with positions and spin orientations of the magnetic atoms, and this will be used rather than a diffpy.Structure object. The structural information is used in class methods that automatically generate the atomic positions and corresponding spin vectors, which can then be stored in a MagStructure object. There are no limits on the number of propagation and basis vectors, allowing for arbitrarily complex magnetic structures. MagSpecies objects can be loaded directly into a MagStructure object or created from inside the MagStructure object. Although a magSpecies object is not required to calculate the mPDF, it will usually provide the easiest way to populate the atom and spin arrays that are required for the mPDF. The reason for having separate MagStructure and MagSpecies classes is that a single magnetic structure may contain multiple magnetic species.

### MPDFcalculator class

The MPDFcalculator class handles the details of the calculation and contains information that is not directly related to the magnetic structure, such as the real-space calculation range, any damping or broadening profile to be applied to the mPDF, scale factors, and experimental parameters like q<sub>min</sub> and q<sub>max</sub>. It requires a MagStructure object as input, from which it extracts the atomic positions and spin vectors to be used in calculating the mPDF. The mPDFcalculator class has options for calculating both the properly normalized and the unnormalized mPDF (the unnormalized mPDF is frequently what we obtain experimentally).

## Simple example: Calculating the mPDF from a pair of spins

We will now create a very simple magnetic structure consisting of just two spins and then calculate the corresponding mPDF.

In [ ]:
### Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from diffpy.mpdf import *
from diffpy.Structure import loadStructure

### Set all plots to be inline
%matplotlib notebook

In [ ]:
### Create a MagStructure object
mstruc = MagStructure()

### Create two atoms in the structure
mstruc.atoms = np.array([[0,0,0],[4,0,0]])

### Create two spin vectors corresponding to the atoms. Let's make them antiferromagnetic.
S=np.array([0,0,1])
mstruc.spins = np.array([S,-S])

### Create the MPDFcalculator object and load the magnetic structure into it
mc = MPDFcalculator(mstruc)

### Calculate and plot the mPDF!
r,fr = mc.calc() # Use calc() if you want to extract the numerical results of the calculation
mc.plot() # Use plot() if you just want to plot the mPDF without extracting the numerical arrays.

Note that the negative peak at 4 Angstroms is due to the antiferromagnetic orientation of the spins. The sloping baseline at low-r is real and is a unique feature of the mPDF that is not found in the atomic PDF.

Just for fun, let's now make a copy of this structure but make the dimer ferromagnetic instead of antiferromagnetic.

In [ ]:
### make a copy of the magnetic structure
mstruc2 = mstruc.copy()

### Set the spins to be ferromagnetic
mstruc2.spins = np.array([S,S])

### Create another mPDF calculator
mc2 = MPDFcalculator(mstruc2)

### Calculate the mPDF
r2, fr2 = mc2.calc()

### Compare the antiferromagnetic and ferromagnetic mPDFs
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r,fr,'b-',label='Antiferro')
ax.plot(r2,fr2,'r-',label='Ferro')

ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'f ($\AA^{-2}$)')

plt.legend()

plt.show()

In [ ]:
plt.close('all')

## Creating a magnetic structure from a CIF file using diffpy.Structure

This example will show how to use the MagSpecies class and additional features in the MagStructure class to quickly generate the atomic positions and spin vectors from a diffpy.Structure object. It will also show how to calculate the unnormalized mPDF, which is useful for comparison to and refinement against neutron total scattering data.

### Load the atomic structure from the CIF file

In [ ]:
### Now let's create a diffpy.Structure object from a CIF file for MnO. This has all the atomic information,
### but none of the magnetic information. Note that a .stru file generated from PDFgui or diffpy.Structure
### would also work just as well.
mno = loadStructure('MnO_cubic.cif')

In [ ]:
### As a refresher, let's take a look at the mno structure object.
print mno

### Create the MagSpecies object

In [ ]:
### We will now create a MagSpecies object to extend the mno structure object.
mspec = MagSpecies()

### Load the mno structure and give the magnetic species a label
mspec.struc = mno
mspec.label = 'Mn2+'

### Now we need to tell it which atoms in MnO are magnetic. From the printed list in the previous cell,
### the magnetic Mn ions are the first four positions in the structure, so we provide the corresponding indices.
mspec.magIdxs = [0,1,2,3]

### Now we provide the magnetic propagation and basis vectors, which are known from previous neutron
### diffraction studies of MnO.
k = np.array([0.5,0.5,0.5])
s = np.array([1,-1,0]) # we won't worry about the magnitude of the basis vector for now
mspec.kvecs = np.array([k])
mspec.basisvecs = np.array([s])

### Now we provide information about the magnetic form factor. We tell the MagSpecies object the type of magnetic
### ion, and it looks up magnetic form factor in a table.
mspec.ffparamkey = 'Mn2'

### Create the MagStructure object

In [ ]:
### Now we can create a MagStructure object and load mnoMag into it.
mstruc = MagStructure()
mstruc.loadSpecies(mspec)
### Now we will generate the atomic positions and spins. It is important to do the atoms first, since the
### spins are generated by applying the propagation and basis vectors to the atomic positions. These methods
### use the information contained in the atomic and magnetic structures to generate arrays of atomic positions
### and spin vectors.
mstruc.makeAtoms()
mstruc.makeSpins()

### And we make the magnetic form factor:
mstruc.makeFF()

### View the magnetic structure

In [ ]:
uc = mno[:4].xyz
visatoms = uc.copy()
visatoms = np.concatenate((visatoms,visatoms+np.array([0,0,1]),visatoms+np.array([0,1,0]),visatoms+np.array([0,0,1])))
visatoms = mno.lattice.cartesian(visatoms)
visspins = mstruc.spinsFromAtoms(visatoms,fractional=False)
mstruc.visualize(visatoms,visspins)

In [ ]:
### Now we can create the mPDF calculator and load the magnetic structure
mc = MPDFcalculator()
mc.magstruc = mstruc

### Plot the mPDF
mc.plot()

In [ ]:
plt.close('all')

# Creating a magnetic structure by defining your own unit cell

This example will show you how to create a magnetic structure from a unit cell that you define yourself. You will have to provide the lattice vectors, the positions of the magnetic atoms in the unit cell, and the magnetic moments corresponding to those atoms.

In [ ]:
### Create the magnetic species and turn off the diffpy.structure option.
mspec = MagSpecies(useDiffpyStruc=False)

### Define the lattice vectors of the unit cell. Let's make a cubic unit cell.
a = 4.0
mspec.latVecs = np.array([[a,0,0], [0,a,0], [0,0,a]])

### Define the positions of the magnetic atoms in the unit cell (in fractional coordinates). Let's make
### We'll make a body-centered structure.
mspec.atomBasis = np.array([[0,0,0], [0.5,0.5,0.5]])

### Define the magnetic moments in the same order as the list of atoms.
### Let's make an antiferromagnet.
mspec.spinBasis = np.array([[0,0,1], [0,0,-1]])


### Create the magnetic structure object and load mspec.
mstruc = MagStructure()
mstruc.loadSpecies(mspec)
mstruc.makeAtoms()
mstruc.makeSpins()

### Let's visualize the first unit cell to make sure we have what we expect.
visAtoms = np.array([[0,0,0],[a,0,0],[0,a,0],[0,0,a],[a,a,0],
                     [a,0,a],[0,a,a],[a,a,a],[0.5*a,0.5*a,0.5*a]])
visSpins = mstruc.spinsFromAtoms(visAtoms, fractional=False)
mstruc.visualize(visAtoms, visSpins)

In [ ]:
### Now we can set up the MPDFcalculator and plot the mPDF.
mc = MPDFcalculator(mstruc)
mc.plot()

In [ ]:
plt.close('all')

# Creating a magnetic structure with multiple species: Simple ferrimagnet

We will create a ferrimagnetic structure to illustrate the use of multiple magnetic species within a single magnetic structure. Let's build another antiferromagnetic body-centered cubic structure but with two different spin species, one with a large moment and one with a small moment.

### Make a magnetic species with a large moment

In [ ]:
### Create the first magnetic species and turn off the diffpy.structure option.
mspec1 = MagSpecies(useDiffpyStruc=False)

### Define the lattice vectors of the unit cell. Let's make a cubic unit cell.
a = 4.0
mspec1.latVecs = np.array([[a,0,0],[0,a,0],[0,0,a]])

### Define the atomic position and magnetic moment.
mspec1.atomBasis = np.array([0,0,0])
mspec1.spinBasis = np.array([0,0,1])
mspec1.label = 'big' ### it is necessary to define unique identifying labels when you have multiple species

### Make a magnetic species with a small moment

In [ ]:
### Now make the other species, starting with mspec1 as a template
mspec2 = mspec1.copy()
mspec2.atomBasis = np.array([0.5,0.5,0.5])
mspec2.spinBasis = np.array([0,0,-0.25])
mspec2.label = 'small'

### Create and view the magnetic structure

In [ ]:
### Create the magnetic structure object and load the species.
mstruc = MagStructure()
mstruc.loadSpecies(mspec1)
mstruc.loadSpecies(mspec2)
mstruc.makeAll()

### Again, let's visualize the first unit cell to make sure we have what we expect.
visAtoms = np.array([[0,0,0],[a,0,0],[0,a,0],[0,0,a],[a,a,0],[a,0,a],[0,a,a],[a,a,a],[0.5*a,0.5*a,0.5*a]])
visSpins = mstruc.spinsFromAtoms(visAtoms, fractional=False)
mstruc.visualize(visAtoms, visSpins, showcrystalaxes=True, axesorigin=np.array([-1,-1,-1]))

### Calculate the mPDF

In [ ]:
### Now we can set up the MPDFcalculator and plot the mPDF.
mc = MPDFcalculator(mstruc)

### Important: since we have two different magnetic species, we must be sure that the calculation
### uses an equivalent number of spins from each species as the "center" of the calculation.
### We do this by changing the calcList attribute of the MPDFcalculator, which is a list of the 
### indices of the atoms/spins to be used as the centers for the calculation. To find the starting
### index of each species, use the getSpeciesIdxs method on the magnetic structure.

mc.calcList = mstruc.getSpeciesIdxs().values()
print mc.calcList

### Now we can plot.
mc.plot()

In [ ]:
plt.close('all')

## Creating more complex magnetic structures: One-dimensional spin helix

This example shows how to use multiple magnetic propagation vectors to create a non-collinear magnetic structure--in this case, a one-dimensional spin helix.

In [ ]:
### We will load in a structure from a CIF file and modify it to simulate a 1-D material.
astruc = loadStructure("MnO_cubic.cif")
astruc.lattice.a = 3.0
astruc.lattice.b = 150.0
astruc.lattice.c = 150.0

### Create the magnetic species object.
helix = MagSpecies(astruc)

### Set up the magnetic propagation and basis vectors for a helical spin configuration.
k = np.array([np.sqrt(2)/10,0,0]) # make the period incommensurate with the lattice
helix.kvecs=np.array([k, -k])

Sk = 0.5*(np.array([0,0,1])+0.5j*np.array([0,1,0])) # j is the numpy symbol for the imaginary unit
helix.basisvecs = np.array([Sk,Sk.conj()])

### Populate with atoms and spins.
helix.rmaxAtoms = 70.0
helix.makeAtoms()
helix.makeSpins()
helix.label = 'helix'

### Create the magnetic structure object.
mstruc = MagStructure()
mstruc.loadSpecies(helix)

#### Visualize the spins.
x,y,z = mstruc.atoms.transpose()
mask = np.logical_and(z==0,np.logical_and(y==0,np.abs(x)<30))
visatoms = mstruc.atoms[mask]
visspins = spinsFromAtoms(mstruc, visatoms, fractional=False)
mstruc.visualize(visatoms, visspins)

In [ ]:
### Now plot the mPDF.
mc = MPDFcalculator(mstruc)
mc.rmax=70.0

mc.plot()

In [ ]:
plt.close('all')

## Exploring some of the additional parameters in the mPDF

Now we will take a look at some of the other aspects of the mPDF that can be adjusted, including:
- Whether the normalized mPDF, unnormalized mPDF, or both are calculated (see Acta A REF)
- Qmax to simulate experimental conditions.
- Parameters that dampen and/or broaden the mPDF, simulating the effects of thermal motion and instrumental resolution.
- "Ordered" scale factor corresponding to the magnitude of the locally ordered moment, and "paramagnetic" scale factor which depends only on the magnitude and spatial extent of a single localized moment, not any correlations between moments.
- The r-range for the calculation.

For this, we will create the antiferromagnetic MnO structure again.

### Create the MnO magnetic structure

In [ ]:
### Create the diffpy structure, MagSpecies, and MagStructure
mno = loadStructure('MnO_cubic.cif')

mspec = MagSpecies(mno, magIdxs=[0,1,2,3], label='Mn2+', ffparamkey='Mn2')
mspec.kvecs = np.array([0.5,0.5,0.5])
mspec.basisvecs = np.array([1,-1,0])

mstruc = MagStructure()
mstruc.loadSpecies(mspec)
mstruc.makeAll()

### Calculate the normalized and unnormalized mPDFs

In [ ]:
mc = MPDFcalculator(mstruc)
r, fr = mc.calc() ### by default, just the normalized mPDF is calculated
r, dr = mc.calc(normalized=False) ### setting the normalized arg to False calculates the unnormalized mPDF

### Let's compare the two quantities

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, fr/fr.max(), 'b-', label='Normalized')
ax.plot(r, dr/dr.max(), 'r-', label='Unnormalized')

ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'f ($\AA^{-2}$)')

plt.legend(loc='best')
plt.tight_layout()
plt.show()

### Or you can calculate them both together:
r, fr, dr = mc.calc(both=True)

### Model the effect of finite Qmax

In [ ]:
### Default mPDF calculator
mc = MPDFcalculator(mstruc)
r, fr = mc.calc()

### Adjust qmax to simulate termination ripples
mcQ = mc.copy()
mcQ.qmax = 25.0
rq,frq = mcQ.calc()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, fr, 'b-', label='Default')
ax.plot(rq, frq, 'r-', label='With qmax')
ax.plot(r, frq-fr-35, 'g-', label='Difference')

ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'f ($\AA^{-2}$)')

plt.legend(loc='best')
plt.show()

### Change the intrinisic mPDF peak width

In [ ]:
### Change the intrinsic mPDF peak width to simulate thermal motion
### and/or instrumental limitations.
mcB = mc.copy()
mcB.gaussPeakWidth = 0.15 # in Angstroms; default is 0.1
rB, frB = mcB.calc()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, fr, 'b-', label='Default')
ax.plot(rB, frB, 'r-', label='Broadened')
ax.plot(r,frB - fr - 35, 'g-', label='Difference')
ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'f ($\AA^{-2}$)')

plt.legend(loc='best')
plt.show()

### Introduce a damping envelope

In [ ]:
### Include a damping factor to simulate a finite correlation length
### and/or instrumental resolution effects.
mcD = mc.copy()
mcD.dampRate = 0.075 # in inverse Angstroms
rD, frD = mcD.calc()

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, fr, 'b-', label='Default')
ax.plot(rD, frD, 'r-', label='Damped')
ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'f ($\AA^{-2}$)')

plt.legend(loc='best')
plt.show()

### Change the ordered scale factor

In [ ]:
### Reduce the ordered scale factor, simulating a reduced ordered moment.
mcS = mc.copy()
mcS.ordScale = 0.5*mc.ordScale
rS, frS, drS = mcS.calc(both=True) # both the normalized and unnormalized mPDF

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, dr, 'b-', label='Default')
ax.plot(rS, drS, 'r-', label='Reduced scale factor')
ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'd ($\AA^{-2}$)')

plt.legend(loc='best')
plt.show()

### Change the calculation range

In [ ]:
### Change the r-range for the calculation from 20 Angstroms (default) to 40.
mcL = mc.copy()
mcL.rmax = 40 # in Angstroms

### Since the r-range is longer, we also need to re-generate the atoms and spins
### to fill a sphere of at least radius 40 Angstroms. The code below does this.
mcL.magstruc.species['Mn2+'].rmaxAtoms = 50
mcL.magstruc.makeAll()

rL,frL,drL = mcL.calc(both=True)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(r, dr, 'bo', label='Default')
ax.plot(rL, drL, 'r-', label='Longer r-grid',lw=2)
ax.set_xlabel(r'r ($\AA$)')
ax.set_ylabel(r'd ($\AA^{-2}$)')

plt.legend(loc='best')
plt.show()

In [ ]:
plt.close('all')

# Simple example of an mPDF fit

Here, we will provide an example of doing a very basic mPDF fit to actual experimental data taken on MnO. The initial structural PDF was refined using PDFgui, and now we use the residual of that structural fit as the dataset against which the mPDF will be refined. To keep things simple, we will just refine the scale factors here.

### Extract the refined atomic structure from the PDFgui file

In [ ]:
PDFguiFile = 'MnOpdfgui.ddp'
struc = getStrucFromPDFgui(PDFguiFile,1)

### Make the magneitc structure

In [ ]:
svec = np.array([1.0,-1.0,0.0])
msp = MagSpecies(struc,magIdxs=[0,1,2,3],ffparamkey='Mn2',rmaxAtoms=50,basisvecs=svec,
                kvecs=np.array([0.5,0.5,0.5]),label=str(0))

mstr=MagStructure()
mstr.loadSpecies(msp)
mstr.makeAll()

### Visualize the magnetic structure
uc = struc[:4].xyz
visatoms = uc.copy()
visatoms = np.concatenate((visatoms,visatoms+np.array([0,0,1]),visatoms+np.array([0,1,0]),visatoms+np.array([0,0,1])))
visatoms = struc.lattice.cartesian(visatoms)
visspins = mstr.spinsFromAtoms(visatoms,fractional=False)
mstr.visualize(visatoms,visspins)

### Extract the mPDF data and make the MPDFcalculator

In [ ]:
r,d = getDiffData(PDFguiFile) ### extract the fit residual from the atomic PDF fit done in PDFgui

mc = MPDFcalculator(mstr)
mc.rmin = r.min()
mc.rmax = r.max()

### Refine the magnetic structure (just scale factor and damping rate for now)

In [ ]:
from scipy.optimize import least_squares

def residual(p,ydata):
    oscale,pscale,damp = p
    mc.ordScale = oscale
    mc.paraScale = pscale
    mc.dampRate = damp
    return ydata - mc.calc(both=True)[2]

p0 = [0.1,0.1,0.1]
optimized = least_squares(residual,p0,bounds=[[0,0,0],[10,10,10]],args=(d,))
fit = mc.calc(both=True)[2]

fig=plt.figure()
ax=fig.add_subplot(111)
ax.plot(r,d,'bo',linestyle='none')
ax.plot(r,fit,'r-',lw=2)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('D($\AA^{-2}$)')

plt.show()    

print optimized.x

### Plot the atomic refinement along with the magnetic refinement

In [ ]:
from diffpy.pdfgui import tui ### useful library to interact with PDFgui projects
prj = tui.LoadProject(PDFguiFile)
fitObj = prj.getFits()[0]
dataSet = fitObj.getDataSet(0)
gobs = np.array(dataSet.Gobs)
robs = np.array(dataSet.robs)
mask = np.logical_and(robs>r.min()-0.001,robs<r.max()+0.001)
gobs = gobs[mask]
gcalc = np.array(dataSet.Gcalc)
mdiff = fit - d

offset1 = 1.2*gobs.min()
offset2 = offset1 - 8*mdiff.max()

# Make the figure.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(r,gobs,marker='o',mfc='none',mec='b',linestyle='none') ### total observed PDF pattern 
ax.plot(r,gcalc,'r-',lw=2) ### calculated atomic PDF
ax.plot(r,d+offset1,marker='o',mfc='none',mec='b',linestyle='none') ### residual from atomic PDF fit
ax.plot(r,fit+offset1,'r-',lw=2) ### calculated mPDF
ax.plot(r,np.zeros_like(r)+offset2,'k:')
ax.plot(r,mdiff+offset2,'g-') ### overall fit residual after including atomic and magnetic PDFs
ax.set_xlim(xmin=0,xmax=mc.rmax)
ax.set_xlabel('r ($\AA$)')
ax.set_ylabel('G, d ($\AA^{-2}$)')

plt.show()

### Compare the magnetic and nuclear scale factors to determine the moment size

If we call $A$ the nuclear scale factor, $B$ the mPDF ordered scale factor, $n_s$ the fraction of atoms in the system that are magnetic, and $\langle b \rangle$ the average nuclear scattering length, then the magnitude of the refined moment $\mathrm{S_{fit}}$ is related to the nominal magnitude of the spin vectors $\mathrm{S_{nom}}$ used in the magnetic structure through the equation $\mathrm{S_{fit}} = \sqrt{\frac{\mathrm{B}\langle b \rangle ^2}{\mathrm{A} n_s}} \mathrm{S_{nom}}$. When multiplied by the g factor (e.g. 2 for pure spins without any orbital contributions to the moment), this gives the ordered moment in Bohr magnetons.

In [ ]:
nucScale = 0.9518 ### refined nuclear scale factor taken from PDFgui

### calculate the average nuclear scattering length
import periodictable as pt ### useful python package included in your installation of diffpy.mpdf
bMn = pt.Mn.neutron.b_c
bO = pt.O.neutron.b_c
bAvg = 0.5*bMn+0.5*bO

### define the ratio of magnetic atoms to total atoms in MnO
ns = 0.5

### determine the nominal spin quantum number from your structure
Snom = np.linalg.norm(mstr.spins[0])

### calculate the refined value of the spin quantum number
Sfit = np.sqrt(mc.ordScale*bAvg**2/(nucScale*ns))*Snom
print Sfit

This is fairly consistent with our expectation of S = 5/2 for Mn2+.

In [ ]:
plt.close('all')

### This concludes the introductory tutorial for diffpy.mpdf. Try out the other example scripts to gain more familiarity with the package, such as how to do a simultaneous co-refinement of the PDF and mPDF. Then go on and use these tools for your own data!